In [ ]:
!wget https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip

--2021-05-03 22:28:50--  https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20194537 (19M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  19.26M  42.8MB/s    in 0.4s    

2021-05-03 22:28:52 (42.8 MB/s) - ‘data.zip’ saved [20194537/20194537]



In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/cats/
  inflating: data/cats/cat.1.jpg     
  inflating: data/cats/cat.10.jpg    
  inflating: data/cats/cat.100.jpg   
  inflating: data/cats/cat.101.jpg   
  inflating: data/cats/cat.102.jpg   
  inflating: data/cats/cat.103.jpg   
  inflating: data/cats/cat.104.jpg   
  inflating: data/cats/cat.105.jpg   
  inflating: data/cats/cat.106.jpg   
  inflating: data/cats/cat.107.jpg   
  inflating: data/cats/cat.108.jpg   
  inflating: data/cats/cat.109.jpg   
  inflating: data/cats/cat.11.jpg    
  inflating: data/cats/cat.110.jpg   
  inflating: data/cats/cat.111.jpg   
  inflating: data/cats/cat.112.jpg   
  inflating: data/cats/cat.113.jpg   
  inflating: data/cats/cat.114.jpg   
  inflating: data/cats/cat.115.jpg   
  inflating: data/cats/cat.116.jpg   
  inflating: data/cats/cat.117.jpg   
  inflating: data/cats/cat.118.jpg   
  inflating: data/cats/cat.119.jpg   
  inflating: data/cats/cat.12.jpg    
  inflating: data/cats/cat

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
print(len(os.listdir('./data/Humans/')))
print(len(os.listdir('./data/cats/')))
print(len(os.listdir('./data/dogs/')))
print(len(os.listdir('./data/horses/')))

202
202
202
202


In [ ]:
path_humans = './data/Humans'
path_cats = './data/cats'
path_dogs = './data/dogs/'
path_horses = './data/horses/'

humans = os.listdir(path_humans)
cats = os.listdir(path_cats)
dogs = os.listdir(path_dogs)
horses = os.listdir(path_horses)

human_image_sizes = []
cats_image_sizes = []
dogs_image_sizes = []
horses_image_sizes = []

for i in range(202):
  hu = plt.imread(os.path.join(path_humans, humans[i]))
  c = plt.imread(os.path.join(path_cats, cats[i]))
  d = plt.imread(os.path.join(path_dogs, dogs[i]))
  ho = plt.imread(os.path.join(path_horses, horses[i]))
  human_image_sizes.append(hu.shape[:-1])
  cats_image_sizes.append(c.shape[:-1])
  dogs_image_sizes.append(d.shape[:-1])
  horses_image_sizes.append(ho.shape[:-1])


In [ ]:


mean_sizes = [np.mean(human_image_sizes, axis=0), np.mean(cats_image_sizes, axis=0),
              np.mean(dogs_image_sizes, axis=0), np.mean(horses_image_sizes, axis=0)]

target_size = np.array(mean_sizes).mean(axis=0)
target_size

array([304.6200495 , 329.92574257])

In [ ]:
image_generator = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

In [ ]:
train_data = image_generator.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')

val_data = image_generator.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

Found 728 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

image_size = [304,329,3]

In [ ]:
input = Input(shape = image_size)
model = ResNet50(input_tensor=input,)

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
  layer.trainable = False


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")

In [ ]:
my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [ ]:
model.fit(train_data, validation_data=val_data, epochs=50, callbacks=my_callbacks)

Epoch 1/50
182/182 [==============================] - 57s 121ms/step - loss: 1.4652 - accuracy: 0.2517 - val_loss: 1.4742 - val_accuracy: 0.2500
Epoch 2/50
182/182 [==============================] - 21s 117ms/step - loss: 1.3781 - accuracy: 0.3337 - val_loss: 1.2757 - val_accuracy: 0.4875
Epoch 3/50
182/182 [==============================] - 21s 116ms/step - loss: 1.3508 - accuracy: 0.3385 - val_loss: 1.2878 - val_accuracy: 0.3500
Epoch 4/50
182/182 [==============================] - 21s 116ms/step - loss: 1.3309 - accuracy: 0.3420 - val_loss: 1.2021 - val_accuracy: 0.4250
Epoch 5/50
182/182 [==============================] - 21s 116ms/step - loss: 1.2869 - accuracy: 0.3843 - val_loss: 1.2555 - val_accuracy: 0.4750
Epoch 6/50
182/182 [==============================] - 21s 116ms/step - loss: 1.2921 - accuracy: 0.3871 - val_loss: 1.1813 - val_accuracy: 0.4750
Epoch 7/50
182/182 [==============================] - 21s 116ms/step - loss: 1.2622 - accuracy: 0.4090 - val_loss: 1.1509 - val_ac

In [ ]:
model.history.history

{'accuracy': [0.2609890103340149,
  0.3461538553237915,
  0.3461538553237915,
  0.34890109300613403,
  0.37637361884117126,
  0.41208791732788086,
  0.40247252583503723,
  0.45467033982276917,
  0.4258241653442383,
  0.4381868243217468,
  0.450549453496933,
  0.41620880365371704],
 'loss': [1.4375436305999756,
  1.3589489459991455,
  1.3363821506500244,
  1.3323274850845337,
  1.3029578924179077,
  1.2598556280136108,
  1.27073335647583,
  1.2102971076965332,
  1.2385705709457397,
  1.2205809354782104,
  1.2042971849441528,
  1.1904815435409546],
 'val_accuracy': [0.25,
  0.48750001192092896,
  0.3499999940395355,
  0.42500001192092896,
  0.4749999940395355,
  0.4749999940395355,
  0.4625000059604645,
  0.48750001192092896,
  0.4375,
  0.375,
  0.44999998807907104,
  0.4749999940395355],
 'val_loss': [1.474163293838501,
  1.2756736278533936,
  1.2878062725067139,
  1.202051043510437,
  1.2554868459701538,
  1.181317687034607,
  1.1509037017822266,
  1.1629788875579834,
  1.166888475418